<a href="https://colab.research.google.com/github/SILVIAIRENE/Data-Scientist-Machine-Learning-Engineer-Introductory-Course/blob/master/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding, SimpleRNN, GRU, LSTM
from keras.datasets import imdb
import pandas as pd
import time

# -------------------
# Hyperparameters
# -------------------
max_features = 20000   # number of words to consider as features
maxlen = 80            # cut texts after this number of words
batch_size = 32
embedding_dim = 128
epochs = 5             # you can increase for better results

# -------------------
# Load data
# -------------------
print("Loading IMDb dataset...")
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)

print(f"Train shape: {x_train.shape}, Test shape: {x_test.shape}")

# -------------------
# Helper function
# -------------------
def build_and_train(model_type="LSTM", units=128):
    print(f"\n=== Training {model_type} ===")
    model = Sequential()
    model.add(Embedding(max_features, embedding_dim, input_length=maxlen))

    if model_type == "SimpleRNN":
        model.add(SimpleRNN(units, dropout=0.2, recurrent_dropout=0.2))
    elif model_type == "GRU":
        model.add(GRU(units, dropout=0.2, recurrent_dropout=0.2))
    elif model_type == "LSTM":
        model.add(LSTM(units, dropout=0.2, recurrent_dropout=0.2))
    else:
        raise ValueError("Unknown model type")

    model.add(Dense(1, activation="sigmoid"))

    model.compile(loss="binary_crossentropy",
                  optimizer="adam",
                  metrics=["accuracy"])

    start = time.time()
    history = model.fit(x_train, y_train,
                        batch_size=batch_size,
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        verbose=2)
    end = time.time()

    score, acc = model.evaluate(x_test, y_test, batch_size=batch_size, verbose=0)
    print(f"{model_type} Test accuracy: {acc:.4f} (time: {end-start:.1f}s)")
    return model_type, acc, end-start

# -------------------
# Run all models
# -------------------
results = []
for m in ["SimpleRNN", "GRU", "LSTM"]:
    name, acc, runtime = build_and_train(m)
    results.append((name, acc, runtime))

# -------------------
# Results table
# -------------------
df = pd.DataFrame(results, columns=["Model", "Accuracy", "Training Time (s)"])
print("\n=== Comparison ===")
print(df.to_string(index=False))


Loading IMDb dataset...
17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Train shape: (25000, 80), Test shape: (25000, 80)

=== Training SimpleRNN ===


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/5
782/782 - 60s - 76ms/step - accuracy: 0.5471 - loss: 0.6834 - val_accuracy: 0.5946 - val_loss: 0.6636
Epoch 2/5
782/782 - 65s - 83ms/step - accuracy: 0.7218 - loss: 0.5439 - val_accuracy: 0.7593 - val_loss: 0.5005
Epoch 3/5
782/782 - 75s - 95ms/step - accuracy: 0.8017 - loss: 0.4445 - val_accuracy: 0.6760 - val_loss: 0.5902
Epoch 4/5
782/782 - 56s - 72ms/step - accuracy: 0.8267 - loss: 0.4011 - val_accuracy: 0.7724 - val_loss: 0.5075
Epoch 5/5
782/782 - 54s - 69ms/step - accuracy: 0.8314 - loss: 0.3913 - val_accuracy: 0.7344 - val_loss: 0.5510
SimpleRNN Test accuracy: 0.7344 (time: 309.9s)

=== Training GRU ===
Epoch 1/5
782/782 - 192s - 245ms/step - accuracy: 0.7695 - loss: 0.4736 - val_accuracy: 0.8417 - val_loss: 0.3724
Epoch 2/5
782/782 - 199s - 254ms/step - accuracy: 0.8930 - loss: 0.2672 - val_accuracy: 0.8538 - val_loss: 0.3508
Epoch 3/5
782/782 - 201s - 257ms/step - accuracy: 0.9417 - loss: 0.1575 - val_accuracy: 0.8400 - val_loss: 0.4075
Epoch 4/5
782/782 - 203s - 26

[Problema 2] (Tarea avanzada) Comparación entre múltiples conjuntos de datos
Experimente con otros conjuntos de datos.

Documentación del conjunto de datos de Keras

Un conjunto de datos de lenguaje natural fácil de usar en Keras es Reuters Newswire Topics Classification.

In [ ]:
from keras.datasets import reuters
from tensorflow.keras.utils import to_categorical
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding, SimpleRNN, GRU, LSTM
import pandas as pd
import time

# Use the same hyperparameters as IMDb
max_features = 20000
maxlen = 80
batch_size = 32
embedding_dim = 128
epochs = 5

# Load and preprocess Reuters dataset
print("Loading Reuters dataset...")
(x_train_reuters, y_train_reuters), (x_test_reuters, y_test_reuters) = reuters.load_data(num_words=max_features)
x_train_reuters = sequence.pad_sequences(x_train_reuters, maxlen=maxlen)
x_test_reuters = sequence.pad_sequences(x_test_reuters, maxlen=maxlen)

# Prepare labels for multi-class classification
num_classes = max(y_train_reuters) + 1
y_train_reuters = to_categorical(y_train_reuters, num_classes)
y_test_reuters = to_categorical(y_test_reuters, num_classes)

print(f"Train shape: {x_train_reuters.shape}, Test shape: {x_test_reuters.shape}")
print(f"Number of classes: {num_classes}")

# Helper function for Reuters (multi-class classification)
def build_and_train_reuters(x_train, y_train, x_test, y_test, model_type="LSTM", units=128):
    print(f"\n=== Training {model_type} on Reuters ===")
    model = Sequential()
    model.add(Embedding(max_features, embedding_dim, input_length=maxlen))

    if model_type == "SimpleRNN":
        model.add(SimpleRNN(units, dropout=0.2, recurrent_dropout=0.2))
    elif model_type == "GRU":
        model.add(GRU(units, dropout=0.2, recurrent_dropout=0.2))
    elif model_type == "LSTM":
        model.add(LSTM(units, dropout=0.2, recurrent_dropout=0.2))
    else:
        raise ValueError("Unknown model type")

    # Multi-class classification requires softmax and categorical_crossentropy
    model.add(Dense(num_classes, activation="softmax"))

    model.compile(loss="categorical_crossentropy",
                  optimizer="adam",
                  metrics=["accuracy"])

    start = time.time()
    history = model.fit(x_train, y_train,
                        batch_size=batch_size,
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        verbose=2)
    end = time.time()

    score, acc = model.evaluate(x_test, y_test, batch_size=batch_size, verbose=0)
    print(f"{model_type} Test accuracy: {acc:.4f} (time: {end-start:.1f}s)")
    return model_type, acc, end-start

# Run all models on Reuters
results_reuters = []
for m in ["SimpleRNN", "GRU", "LSTM"]:
    name, acc, runtime = build_and_train_reuters(x_train_reuters, y_train_reuters,
                                                 x_test_reuters, y_test_reuters,
                                                 model_type=m)
    results_reuters.append((name, acc, runtime))

# Results table
df_reuters = pd.DataFrame(results_reuters, columns=["Model", "Accuracy", "Training Time (s)"])
print("\n=== Reuters Results ===")
print(df_reuters.to_string(index=False))

# Compare with IMDb results (if you want to show both)
print("\n=== Comparison: IMDb vs Reuters ===")
print("IMDb (Binary Classification):")
# Your IMDb results from before
imdb_results = [
    ("SimpleRNN", 0.7852, 48.9),
    ("GRU", 0.8174, 1020.1),
    ("LSTM", 0.8257, 1114.6)
]
df_imdb = pd.DataFrame(imdb_results, columns=["Model", "Accuracy", "Training Time (s)"])
print(df_imdb.to_string(index=False))

print("\nReuters (Multi-class Classification):")
print(df_reuters.to_string(index=False))

Loading Reuters dataset...
2110848/2110848 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Train shape: (8982, 80), Test shape: (2246, 80)
Number of classes: 46

=== Training SimpleRNN on Reuters ===


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/5
281/281 - 29s - 102ms/step - accuracy: 0.3192 - loss: 2.5671 - val_accuracy: 0.3620 - val_loss: 2.4313
Epoch 2/5
281/281 - 33s - 118ms/step - accuracy: 0.3840 - loss: 2.3238 - val_accuracy: 0.4069 - val_loss: 2.2335
Epoch 3/5
281/281 - 16s - 55ms/step - accuracy: 0.4557 - loss: 2.1199 - val_accuracy: 0.4199 - val_loss: 2.2057
Epoch 4/5
281/281 - 17s - 59ms/step - accuracy: 0.5073 - loss: 1.9455 - val_accuracy: 0.4225 - val_loss: 2.2317
Epoch 5/5
281/281 - 17s - 59ms/step - accuracy: 0.5536 - loss: 1.7760 - val_accuracy: 0.4297 - val_loss: 2.2785
SimpleRNN Test accuracy: 0.4297 (time: 110.9s)

=== Training GRU on Reuters ===
Epoch 1/5
281/281 - 59s - 209ms/step - accuracy: 0.4393 - loss: 2.1111 - val_accuracy: 0.5227 - val_loss: 1.7998
Epoch 2/5
281/281 - 82s - 291ms/step - accuracy: 0.5600 - loss: 1.6867 - val_accuracy: 0.5641 - val_loss: 1.7338
Epoch 3/5
281/281 - 83s - 297ms/step - accuracy: 0.6142 - loss: 1.4927 - val_accuracy: 0.6042 - val_loss: 1.5513
Epoch 4/5
281/281 -

[Problema 3] Explicación de otras clases
Hay otras clases relacionadas en la documentación. Explíquelas. Algunas de estas clases rara vez se usan en la práctica.

Enfermera registrada
SimpleRNNCell
GRUCell
Celda LSTMCell
Células RNN apiladas
CuDNNGRU
CuDNNNLSTM
prueba

In [ ]:
from keras.models import Sequential
from keras.layers import RNN, SimpleRNNCell, GRUCell, LSTMCell, StackedRNNCells, Dense, Embedding
from keras.datasets import imdb
from keras.preprocessing import sequence

max_features = 20000
maxlen = 80
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)

# Example 1: Using SimpleRNNCell inside RNN
model1 = Sequential([
    Embedding(max_features, 128),
    RNN(SimpleRNNCell(32)),
    Dense(1, activation="sigmoid")
])

# Example 2: Using GRUCell
model2 = Sequential([
    Embedding(max_features, 128),
    RNN(GRUCell(32)),
    Dense(1, activation="sigmoid")
])

# Example 3: Using LSTMCell
model3 = Sequential([
    Embedding(max_features, 128),
    RNN(LSTMCell(32)),
    Dense(1, activation="sigmoid")
])

# Example 4: Stacked cells (2 LSTM layers in one RNN)
stacked_cells = StackedRNNCells([LSTMCell(32), LSTMCell(32)])
model4 = Sequential([
    Embedding(max_features, 128),
    RNN(stacked_cells),
    Dense(1, activation="sigmoid")
])

model1
model2
model3
model4

17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


<Sequential name=sequential_3, built=False>